In [4]:
import tweepy
import csv
import pandas as pd
import re

In [8]:
consumer_key = ' '
consumer_secret = ' '
access_token = ' '
access_token_secret = ' '

In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

csvFile = open('marriott_tweets.csv', 'w')

csvWriter = csv.writer(csvFile)

tweets = []
csvWriter.writerow(["timestamp","tweet"])
for tweet in tweepy.Cursor(api.search,q="#marriott",lang="en").items():
    tweets.append((tweet.created_at, tweet.text))
    csvWriter.writerow([tweet.created_at, tweet.text.strip().encode('utf-8')])

In [35]:
sentiment_scores = {}

with open ("tidytext_sentiments.txt",'r') as input_file :
    next(input_file)
    for line in input_file.readlines():     
        word, sentiment, lexicon = line.split("\t")
        if sentiment == "positive" :
            sentiment_scores[word] = 1
        elif sentiment == "negative" :
            sentiment_scores[word] = -1 

assert(len(sentiment_scores) > 0)

In [49]:
running_score = 0
scores = []
for idx, (date, content) in enumerate(tweets1) :
    for word in content.split():
        if word.lower() in sentiment_scores :
            running_score += sentiment_scores[word.lower()]
    
    scores.append((date, content, running_score))
print(scores)

[(datetime.datetime(2018, 12, 7, 16, 36, 25), 'Happy Friday and Happy Holidays!\n\n#courtyardwilloughby #courtyardmarriott #marriott #chrismasdecor #happyholidays… https://t.co/XfIXHoJKTq', 2), (datetime.datetime(2018, 12, 7, 16, 36, 18), '"What lessons can we learn from #Marriott’s response to their #CyberBreach?" #CyberSecurity #databreach… https://t.co/mLNErClLmp', 3), (datetime.datetime(2018, 12, 7, 16, 33, 5), '#Dundee: “The capital plan includes significant investment (including) more than £20 million for the construction o… https://t.co/6seMU99Uc8', 4), (datetime.datetime(2018, 12, 7, 16, 11, 3), 'Marriott, now the largest hotel chain in the world, said Starwood’s guest reservation database was compromised by u… https://t.co/SWFmf94kgh', 4), (datetime.datetime(2018, 12, 7, 16, 11, 2), '#Marriott to reimburse some guests for new #passports after massive #databreach https://t.co/66lrafUqBo #infosec #cyberattacks', 5), (datetime.datetime(2018, 12, 7, 16, 9, 39), 'RT @ShaziaAttaMarr

In [50]:
df = pd.DataFrame(scores,columns=['Timestamp','Tweet','Sentiment'])
print(df)

               Timestamp                                              Tweet  \
0    2018-12-07 16:36:25  Happy Friday and Happy Holidays!\n\n#courtyard...   
1    2018-12-07 16:36:18  "What lessons can we learn from #Marriott’s re...   
2    2018-12-07 16:33:05  #Dundee: “The capital plan includes significan...   
3    2018-12-07 16:11:03  Marriott, now the largest hotel chain in the w...   
4    2018-12-07 16:11:02  #Marriott to reimburse some guests for new #pa...   
5    2018-12-07 16:09:39  RT @ShaziaAttaMarri: What a shame #Marriott Is...   
6    2018-12-07 16:08:57  The international hotel chain, Marriott, has a...   
7    2018-12-07 16:07:44  RT @CorboyDemetrio: Clues suggest China may ha...   
8    2018-12-07 16:02:48  The #Marriott hack fall-out continues to make ...   
9    2018-12-07 16:02:32  What Marriott’s Data Breach Means for the Hote...   
10   2018-12-07 16:01:09  #ICYMI "Why you should #freeze your #credit, e...   
11   2018-12-07 15:46:22  I Just Stayed at One of #M

In [51]:
df["Tweet"] = df["Tweet"].str.replace(r'RT @'," ",regex=True)
df["Tweet"] = df["Tweet"].str.replace(r'#'," ",regex=True)
df["Tweet"] = df["Tweet"].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
print(df["Tweet"])

0       Happy Friday and Happy Holidays!\n\n courtyard...
1       "What lessons can we learn from  Marriott’s re...
2        Dundee: “The capital plan includes significan...
3       Marriott, now the largest hotel chain in the w...
4        Marriott to reimburse some guests for new  pa...
5        ShaziaAttaMarri: What a shame  Marriott Islam...
6       The international hotel chain, Marriott, has a...
7        CorboyDemetrio: Clues suggest China may have ...
8       The  Marriott hack fall-out continues to make ...
9       What Marriott’s Data Breach Means for the Hote...
10       ICYMI "Why you should  freeze your  credit, e...
11      I Just Stayed at One of  Marriott's New Hotels...
12       NOVAMIntl: Compromised machines lead to other...
13       sang_alertboot:  Marriott’s breach response i...
14       Marriott  Starwood  Data  Breach Highlights  ...
15             Marriott looking at  China in  databreach 
16      DIGITAL SALES MASTERY BY @spencerlodge \nMASTE...
17       Marri

In [53]:
df["Tweet"] = df["Tweet"].str.lower()
print(df["Tweet"])

0       happy friday and happy holidays!\n\n courtyard...
1       "what lessons can we learn from  marriott’s re...
2        dundee: “the capital plan includes significan...
3       marriott, now the largest hotel chain in the w...
4        marriott to reimburse some guests for new  pa...
5        shaziaattamarri: what a shame  marriott islam...
6       the international hotel chain, marriott, has a...
7        corboydemetrio: clues suggest china may have ...
8       the  marriott hack fall-out continues to make ...
9       what marriott’s data breach means for the hote...
10       icymi "why you should  freeze your  credit, e...
11      i just stayed at one of  marriott's new hotels...
12       novamintl: compromised machines lead to other...
13       sang_alertboot:  marriott’s breach response i...
14       marriott  starwood  data  breach highlights  ...
15             marriott looking at  china in  databreach 
16      digital sales mastery by @spencerlodge \nmaste...
17       marri

In [54]:
df.to_csv("clean_tweets.csv", sep=',', encoding='utf-8')

In [27]:
import pandas as pd
marriott_df = pd.read_csv("marriott_clean_tweets.csv",index_col=0)
marriott_df.dropna(inplace=True)

hilton_df = pd.read_csv("hilton_clean_tweets.csv",index_col=0)
hilton_df.dropna(inplace=True)

,Timestamp,Tweet,Sentiment
0,2018-12-07 16:36:25,happy friday and happy holidays!\r\n\r\n court...,2
1,2018-12-07 16:36:18,"""what lessons can we learn from marriott’s re...",3
2,2018-12-07 16:33:05,dundee: “the capital plan includes significan...,4
3,2018-12-07 16:11:03,"marriott, now the largest hotel chain in the w...",4
4,2018-12-07 16:11:02,marriott to reimburse some guests for new pa...,5
5,2018-12-07 16:09:39,shaziaattamarri: what a shame marriott islam...,4
6,2018-12-07 16:08:57,"the international hotel chain, marriott, has a...",3
7,2018-12-07 16:07:44,corboydemetrio: clues suggest china may have ...,2
8,2018-12-07 16:02:48,the marriott hack fall-out continues to make ...,1
9,2018-12-07 16:02:32,what marriott’s data breach means for the hote...,0


In [182]:
from nltk.corpus import stopwords
from string import punctuation

def tweet_features(the_tweet) :
    """ Input: A twitter description
        Output: A dictionary listin the words that are in 
                the description.
                
        This function does some cleaning on the descriptions,
        removing some punctuation, splitting on whitespace, 
        dropping to lower case. It returns a dictionary 
        of the form 
            {example : True,
             word :    True}
    
        """
    exclude = set(punctuation)
    exclude.remove("#") #useful for twitter...
    exclude_words = ["marriott","hilton","marriotts","hiltons","hiltonhotels","doubletree"]
    stopwords = nltk.corpus.stopwords.words('english')
    exclude_words.extend(stopwords)
    
    the_tweet = ''.join([ch.lower() for ch in the_tweet if ch not in exclude])
    
    word_list = the_tweet.split()

    ret_val = {}
    
    for word in word_list:
        if word not in exclude_words:
            ret_val[word] = True
    
    return(ret_val)   

In [183]:
marriott_tweets = [(tweet,"marriott")  for tweet in marriott_df.Tweet]
hilton_tweets = [(tweet,"hilton")  for tweet in hilton_df.Tweet]
tweets = marriott_tweets + hilton_tweets
tweets

[('happy friday and happy holidays!\r\n\r\n courtyardwilloughby  courtyardmarriott  marriott  chrismasdecor  happyholidays… ',
  'marriott'),
 ('"what lessons can we learn from  marriott’s response to their  cyberbreach?"  cybersecurity  databreach… ',
  'marriott'),
 (' dundee: “the capital plan includes significant investment (including) more than £20 million for the construction o… ',
  'marriott'),
 ('marriott, now the largest hotel chain in the world, said starwood’s guest reservation database was compromised by u… ',
  'marriott'),
 (' marriott to reimburse some guests for new  passports after massive  databreach ',
  'marriott'),
 (' shaziaattamarri: what a shame  marriott islamabad!  accessibilityforall ',
  'marriott'),
 ('the international hotel chain, marriott, has announced a massive data hack that impacts as many as 500 million cust… ',
  'marriott'),
 (' corboydemetrio: clues suggest china may have been behind the massive  marriott hack affecting 500 million people ',
  '

In [184]:
import random
random.shuffle(tweets)

In [200]:
import nltk
test_set_size = 5000

featuresets = [(tweet_features(tweet), label) for (tweet, label) in tweets]
train_set, test_set = featuresets[test_set_size:], featuresets[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [201]:
print(nltk.classify.accuracy(classifier, test_set))

0.8922


In [202]:
classifier.show_most_informative_features(20)

Most Informative Features
                   drive = True           hilton : marrio =    150.0 : 1.0
                     dec = True           hilton : marrio =    144.5 : 1.0
                     yes = True           hilton : marrio =    144.5 : 1.0
                      10 = True           hilton : marrio =    144.5 : 1.0
                tomorrow = True           hilton : marrio =     35.5 : 1.0
                 hilton… = True           hilton : marrio =     35.5 : 1.0
                   water = True           hilton : marrio =     35.5 : 1.0
                   front = True           hilton : marrio =     26.9 : 1.0
                     san = True           hilton : marrio =     26.4 : 1.0
                    chef = True           hilton : marrio =     24.5 : 1.0
                  hoping = True           hilton : marrio =     24.5 : 1.0
                    home = True           hilton : marrio =     24.5 : 1.0
                 airport = True           hilton : marrio =     20.8 : 1.0